# 要約 
このノートブックは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、チャットボットの応答に対する人間の好みを予測するための機械学習モデルを構築することに取り組んでいます。具体的には、提供されたデータセット内のPythonとTensorFlow、およびKerasを使用して、自然言語処理を行い、複数の異なるモデルの出力を評価し、勝者を予測するシステムを作成します。

### 使用ライブラリと手法
1. **ライブラリのインポート**: `Keras`, `TensorFlow`, `NumPy`, `Pandas`など、モデル構築やデータ処理のために多くのライブラリをインポートします。また、`keras_nlp`を利用して自然言語処理を実行します。
  
2. **ハードウェアの設定**: 利用可能なGPUやTPUを確認し、最適な学習戦略を選択します。これにより、分散学習を利用してトレーニングの効率を向上させます。

3. **データの前処理**: トレーニングデータとテストデータを読み込み、重複を削除するなどして基本的な整形を行います。また、応答をプロンプトに基づいて文脈化してペアを作成し、各応答をテキストクリーニングします。

4. **特徴量の生成**: `DataFrameStatsProcessor`クラスを使用し、データフレームから特徴量を計算し、欠損値の統計や応答の長さに関する情報を整理します。

5. **モデル構築**: `DebertaV3Backbone`を用いたニューラルネットワークモデルを構築し、層を追加して最後はSoftmax関数を用いてクラス確率を予測します。ここでは、L2正則化やドロップアウトレイヤーを使用して過学習を防ぎます。

6. **トレーニング**: 学習率のスケジュールを設定し、モデルをトレーニングします。学習が進むにつれてモデルの性能を評価しつつ、最良のモデルを保存します。

7. **評価と予測**: トレーニングを経たモデルを使用して、テストデータセットに対して予測を実行します。また、FGM（Fast Gradient Method）を利用してモデルの堅牢性を評価します。

8. **出力ファイルの作成**: 最後に、予測結果をCSVファイルとして保存し、コンペティションに提出します。

このノートブックは、全体を通して深層学習を活用した自然言語処理モデルの構築と評価に焦点を当てており、特にモデルの再現性を高めるための工夫が随所に見られます。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する専門用語の解説です。初心者がつまずきそうな要点に焦点を当てています。

1. **Kerasバックエンド (Keras backend)**:
   - Kerasは高レベルのニューラルネットワークAPIであり、TensorFlowやTheanoといった他の深層学習ライブラリの上で動作します。`KERAS_BACKEND`を設定することで、Kerasがどのライブラリを使用するかを指定します。

2. **TPU (Tensor Processing Unit)**:
   - Googleが開発した、特に機械学習のために設計されたハードウェア加速器です。TPUは演算を効率化し、大規模なモデルをより速く訓練または推論するために特化しています。

3. **混合精度 (Mixed Precision)**:
   - 深層学習トレーニング中に、異なる精度（通常はfloat32とfloat16）を使用してメモリ使用量を減少させる手法です。これにより、トレーニングプロセスが速くなり、大きなバッチサイズを使用できるようになります。

4. **データローダー (DataLoader)**:
   - モデルにデータを供給するための仕組みを指します。効率的なデータ読み込みや前処理のために使用され、ハードウェアのキャパシティを最大限に活用するのに役立ちます。

5. **FGM (Fast Gradient Method)**:
   - モデルの堅牢性を向上させる手法の一つで、特徴量に摺動（微小なランダムな変更）を加えることによって、モデルが攻撃に強くなることを目的としています。

6. **AWP (Adversarial Weight Perturbation)**:
   - モデルの重みを動的に微調整することにより、強化学習されたモデルの堅牢性を高める手法です。この手法では、各バッチの開始時に重みに摺動を加えます。

7. **Zスコア正規化 (Z-score Normalization)**:
   - データを標準化する手法で、データの平均を0、標準偏差を1に調整します。これにより、異なるスケールの特徴量を比較可能にします。

8. **前処理器 (Preprocessor)**:
   - データをモデルが扱いやすい形式に変換するための仕組み。トークン化、パディング、エンコーディングなどの操作を行います。

9. **温度スケーリング (Temperature Scaling)**:
   - モデルの予測確率を調整するための手法です。ロジット（ニューラルネットワークの出力）を温度パラメータで割ることによって、確率分布を平滑化し、過剰適合を防ぎます。

10. **ラベルスムージング (Label Smoothing)**:
    - 正解ラベルに対して、ほんの少しノイズを加えて柔軟性を持たせる手法です。これにより、過剰適合を防ぎ、一般化性能を向上させる効果があります。

これらの用語は、実務経験が乏しい初心者にとっては馴染みが薄いものでありながら、深層学習モデルの実装においては非常に重要な要素です。

---


# ライブラリのインポート

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # または "jax" または "torch"
# Kerasのバックエンドとして使用するフレームワークを環境変数に設定します

import re  # 正規表現を操作するためのライブラリをインポートします

import keras_nlp  # Kerasの自然言語処理用ライブラリをインポートします
import keras  # Kerasライブラリをインポートします
import tensorflow as tf  # TensorFlowライブラリをインポートします

import numpy as np  # NumPyライブラリをインポートします（数値計算用）
import pandas as pd  # Pandasライブラリをインポートします（データ操作用）
from tqdm import tqdm  # プログレスバー表示のためのライブラリをインポートします
import json  # JSONデータを扱うためのライブラリをインポートします

# 利用可能なGPUの数

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# 利用可能なGPUの数を出力します

strategy = tf.distribute.MirroredStrategy()
# モデルの分散学習のために、MirroredStrategyを使用します
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
# 同期中のデバイス（GPU）の数を出力します

# TPU（Tensor Processing Unit）

In [ ]:
# ハードウェアを検出し、適切な分散戦略を返します
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPUの検出。TPU_NAME環境変数が設定されていればパラメータは不要です。Kaggleでは常にこの設定が行われています。
    print('Running on TPU ', tpu.master())
    # TPUが見つかった場合、そのマスターの情報を出力します
except ValueError:
    tpu = None  # TPUが見つからない場合、tpuをNoneに設定します

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    # TPUクラスタへの接続を確立します
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # TPUシステムを初期化します
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    # TPU用の分散戦略を設定します
else:
    strategy = tf.distribute.get_strategy()  # TensorFlowのデフォルトの分散戦略を取得します。CPUおよび単一GPUで動作します。

print("REPLICAS: ", strategy.num_replicas_in_sync)
# 同期中の複製数を出力します

# 設定

In [ ]:
class CFG:
    seed = 42  # ランダムシードの設定
    preset = "deberta_v3_extra_small_en"  # 使用するモデルのプリセット
    sequence_length = 512  # 入力シーケンスの最大長
    epochs = 6  # 学習するエポック数
    batch_size = 16  # バッチサイズ
#     batch_size = 16 * strategy.num_replicas_in_sync  # (コメントアウトされたコード) 複製に応じたバッチサイズの計算
    scheduler = 'cosine'  # 学習率スケジューラーの設定
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}  # ラベルと名前のマッピング
    name2label = {v:k for k, v in label2name.items()}  # 名前からラベルへの逆マッピング
    class_labels = list(label2name.keys())  # 使用するクラスラベルのリスト
    class_names = list(label2name.values())  # 使用するクラス名のリスト

# 再現性
ランダムシードの値を設定することで、毎回の実行で類似の結果を得ることができます。

In [ ]:
keras.utils.set_random_seed(CFG.seed)  # Kerasのランダムシードを設定します。これにより、モデルの再現性が確保されます。

# 混合精度

本ノートブックでは、GPUのメモリ使用量を減少させるために、float32精度の代わりに混合精度を使用してトレーニングと推論を行います。これにより、より大きなバッチサイズを使用できるようになり、トレーニングと推論の時間を短縮できます。

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16")
# "mixed_float16"ポリシーを設定します。この設定により、モデルの一部は自動的にfloat16で計算され、他の部分（例えば損失関数の計算）は安定性を保つために引き続きfloat32を使用します。

# データセットのパス

In [ ]:
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'  # データセットのベースパスを設定します。このパスはKaggleの入力データを指します。

# メタデータ
## ファイル

### `train.csv`
- `id`: 各行のユニーク識別子。
- `model_[a/b]`: モデルの識別子。train.csvには存在するが、test.csvには存在しません。
- `prompt`: 両モデルに与えられた入力プロンプト。
- `response_[a/b]`: モデル_[a/b]のプロンプトに対する応答。
- `winner_model_[a/b/tie]`: 審査員の選択を示すバイナリ列（正解ターゲット）。

### `test.csv`
- `id`: 各行のユニーク識別子。
- `prompt`: 両モデルに与えられた入力プロンプト。
- `response_[a/b]`: モデル_[a/b]のプロンプトに対する応答。

In [ ]:
# トレーニングデータを読み込む
df = pd.read_csv(f'{BASE_PATH}/train.csv')  # train.csvファイルを読み込みます
ultrachat_df = pd.read_csv('/kaggle/input/ultrachat-train/ultrachat_s42_a0.5.csv')  # ultrachatデータを読み込みます
df = pd.concat([df, ultrachat_df], axis=0)  # データフレームを縦に結合します
lmsys_33k_deduplicated = pd.read_csv('/kaggle/input/lmsys-33k-deduplicated/lmsys-33k-deduplicated.csv')  # 重複のないlmsysデータを読み込みます
df = pd.concat([df, lmsys_33k_deduplicated], axis=0)  # 再びデータフレームを結合します
# ultrafeedback_lmsysformat = pd.read_parquet('/kaggle/input/ultrafeedback-lmsysformat/ultrafeedback_lmsysformat.parquet', engine='pyarrow')
# ultrafeedback_lmsysformat['prompt'] = ultrafeedback_lmsysformat['prompt'].apply(lambda x: f'["{x}"]')
# df = pd.concat([df, ultrafeedback_lmsysformat], axis=0)  # コメントアウトされたコード。別のデータフレームの読み込みと結合。

# テストデータを読み込む
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')  # test.csvファイルを読み込みます

# display(ultrafeedback_lmsysformat.head())  # コメントアウトされたコード。ultrafeedbackデータの先頭を表示。
display(df.head())  # トレーニングデータフレームの先頭5行を表示します。

In [ ]:
df = df.drop("id", axis=1)  # 'id'列を削除します
df = df.drop_duplicates(keep="first", ignore_index=True)  # 重複行を削除し、インデックスを再設定します

for col in ["prompt"]:
    df[col] = df[col].apply(lambda x: eval(x))  # 'prompt'列の各要素を評価してリストに変換します
    test_df[col] = test_df[col].apply(lambda x: eval(x))  # テストデータの'prompt'列も同様に処理します
for col in ["response_a", "response_b"]:
    df[col] = df[col].apply(lambda x: eval(x.replace("null", "None")))  # 'null'を'None'に置き換え、評価します
    test_df[col] = test_df[col].apply(lambda x: eval(x.replace("null", "None")))  # テストデータも同様に処理します
    
# サンプリングデータ（コメントアウト）
# df = df.sample(frac=0.01)  # データの1%をサンプリングします

# ラベル変換
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)  # 各行で最大値のインデックスを取得し、クラス名を設定
df["class_label"] = df.class_name.map(CFG.name2label)  # クラス名をラベルに変換します

# サンプルを表示
display(df.head())  # トレーニングデータフレームの先頭5行を表示します
# サンプルを表示
display(test_df.head())  # テストデータフレームの先頭5行を表示します

## プロンプトで応答を文脈化する

私たちのアプローチでは、すべての応答に対して単一のプロンプトを使用するのではなく、各応答をプロンプトに基づいて文脈化します。これは、各応答ごとに、モデルに同じプロンプトのセットとその各応答（例えば、「(P + R_A)」、「(P + R_B)」など）を提供することを意味します。

> 一部のプロンプトおよび応答は `utf-8` エンコーディングが使用されていない場合があり、その結果データローダーの作成時にエラーが発生する可能性があります。この場合、これらを空の文字列に置き換えます。

In [ ]:
def make_pairs(row):
    row['options'] = []  # optionsリストを初期化します
    row["encode_fail"] = False  # エンコード失敗フラグを初期化します

    try:
        # 必要なキーがすべてrow辞書に存在することを確認します
        prompts = row['prompt']  # プロンプトを取得します
        responses_a = row['response_a']  # 応答Aを取得します
        responses_b = row['response_b']  # 応答Bを取得します
        
        # リストの長さが一致することを確認します
        if not (len(prompts) == len(responses_a) == len(responses_b)):
            raise ValueError("The lists 'prompt', 'response_a', and 'response_b' must be of the same length.")
            # プロンプト、応答A、および応答Bのリストが同じ長さである必要があります
            
        response_a_str = ''
        response_b_str = ''
        
        for idx in range(len(prompts)):
            response_a_str += f"Prompt: {prompts[idx]}\n\nResponse: {responses_a[idx]}"
            response_b_str += f"Prompt: {prompts[idx]}\n\nResponse: {responses_b[idx]}"
        
        # テキストのクリーニング、たとえば、認識できないUnicode文字を除去または置き換えます
        clean_response_a_str = "".join(filter(lambda x: ord(x) < 128, response_a_str))  # ASCII文字のみを残します
        clean_response_b_str = "".join(filter(lambda x: ord(x) < 128, response_b_str))  # ASCII文字のみを残します
        
        row['options'].append(clean_response_a_str)  # クリーンされた応答Aをoptionsリストに追加
        row['options'].append(clean_response_b_str)  # クリーンされた応答Bをoptionsリストに追加
        
    except KeyError as e:
        print(f"Missing key in row: {e}")  # 指定されたキーが見つからない場合のエラーメッセージ
        row["encode_fail"] = True  # エンコード失敗フラグを立てる
    except ValueError as e:
        print(e)  # 値エラーの場合のメッセージ
        row["encode_fail"] = True  # エンコード失敗フラグを立てる
    except Exception as e:
        # その他すべての例外をキャッチ
        print(f"An unexpected error occurred: {e}")  # 予期しないエラーのメッセージ
        row["encode_fail"] = True  # エンコード失敗フラグを立てる

    return row  # 処理された行を返します

In [ ]:
df = df.apply(make_pairs, axis=1)  # 各行に対してmake_pairs関数を適用します
display(df.head(2))  # トレーニングデータフレームの最初の2行を表示します

test_df = test_df.apply(make_pairs, axis=1)  # テストデータフレームの各行にも同様にmake_pairs関数を適用します
display(test_df.head(2))  # テストデータフレームの最初の2行を表示します

## エンコーディング失敗の統計

In [ ]:
df.encode_fail.value_counts(normalize=False)  # エンコーディング失敗フラグのカウントを表示します（正規化せず）。

# EDA（探索的データ分析）

In [ ]:
class DataFrameStatsProcessor:
    def __init__(self, df):
        self.df = df  # 初期化時にデータフレームを設定します

    def _is_empty(self, string: str) -> bool:
        return bool(re.match("^\s*$", string))  # 文字列が空かどうかを判断します

    def _len(self, string: str) -> int:
        if string is None:
            return 0  # Noneの場合は長さ0を返します
        return len(string)  # 文字列の長さを返します

    def _add_len_stats(self, col: str) -> pd.DataFrame:
        if col == "prompt":
            col_prefix = "p_len"  # プロンプトの列接頭辞
        elif col == "response_a":
            col_prefix = "res_a_len"  # 応答Aの列接頭辞
        elif col == "response_b":
            col_prefix = "res_b_len"  # 応答Bの列接頭辞
        
        # 各長さの統計を計算
        self.df[f"{col_prefix}_sum"] = self.df[col].apply(lambda x: sum(self._len(s) for s in x))
        self.df[f"{col_prefix}_mean"] =  self.df[col].apply(lambda x: np.mean(list(self._len(s) for s in x)))
        self.df[f"{col_prefix}_max"] = self.df[col].apply(lambda x: max(self._len(s) for s in x))
        self.df[f"{col_prefix}_sum_log"] = np.log1p(self.df[f"{col_prefix}_sum"])
        self.df[f"{col_prefix}_mean_log"] =  np.log1p(self.df[f"{col_prefix}_mean"])
        self.df[f"{col_prefix}_max_log"] = np.log1p(self.df[f"{col_prefix}_max"])
        
        return self.df  # 更新されたデータフレームを返します
    
    def z_score_normalize(self, columns):
        """
        指定した列にZスコア正規化を適用します。
        パラメータ:
            columns (list): Zスコア正規化を実施する列名のリスト。
        """
        for col in columns:
            self.df[col] = (self.df[col] - self.df[col].mean()) / self.df[col].std()  # Zスコア正規化を実施します
    
    def process_dataframe(self):
        # データフレームの処理を実施します
        self.df["n_prompts"] = self.df["prompt"].apply(lambda x: len(x))  # プロンプトの数を数えます
        self.df["n_res_a"] = self.df["response_a"].apply(lambda x: len(x))  # 応答Aの数を数えます
        self.df["n_res_b"] = self.df["response_b"].apply(lambda x: len(x))  # 応答Bの数を数えます
        assert ((self.df["n_prompts"] == self.df["n_res_a"]) & (self.df["n_prompts"] == self.df["n_res_b"])).all()  # 数が一致することを確認します

        # プロンプトと応答の欠損値や空の値をカウントします
        self.df["n_na_prompts"] = self.df["prompt"].apply(lambda ps: sum(1 if p is None else 0 for p in ps))
        self.df["n_empty_prompts"] = self.df["prompt"].apply(lambda ps: sum(1 if p is not None and self._is_empty(p) else 0 for p in ps))
        self.df["n_na_res_a"] = self.df["response_a"].apply(lambda ps: sum(1 if p is None else 0 for p in ps))
        self.df["n_empty_res_a"] = self.df["response_a"].apply(lambda ps: sum(1 if p is not None and self._is_empty(p) else 0 for p in ps))
        self.df["n_na_res_b"] = self.df["response_b"].apply(lambda ps: sum(1 if p is None else 0 for p in ps))
        self.df["n_empty_res_b"] = self.df["response_b"].apply(lambda ps: sum(1 if p is not None and self._is_empty(p) else 0 for p in ps))

        # 欠損値の合計を計算します
        self.df["n_miss_res_a"] = self.df["n_na_res_a"] + self.df["n_empty_res_a"]
        self.df["n_miss_res_b"] = self.df["n_na_res_b"] + self.df["n_empty_res_b"]

        # 有効な応答数を計算します
        self.df["n_eff_res_a"] = self.df["n_res_a"] - self.df["n_miss_res_a"]
        self.df["n_eff_res_b"] = self.df["n_res_b"] - self.df["n_miss_res_b"]

        # 長さの統計を追加します
        self._add_len_stats("prompt")
        self._add_len_stats("response_a")
        self._add_len_stats("response_b")

        self.df["res_len_mean_diff"] = self.df["res_a_len_mean"] - self.df["res_b_len_mean"]  # 応答AとBの長さの平均の差を計算
        self.df["res_len_mean_diff_clip"] = self.df["res_len_mean_diff"].clip(-6000, 6000)  # 差の範囲をクリップします

        # プロンプトの欠損値をカウントします
        self.df["n_miss_prompts"] = self.df["n_na_prompts"] + self.df["n_empty_prompts"]
        self.df["n_eff_prompts"] = self.df["n_prompts"] - self.df["n_miss_prompts"]

        # 比率を計算します
        self.df["na_prompt_ratio"] = self.df["n_na_prompts"] / self.df["n_prompts"]
        self.df["empty_prompt_ratio"] = self.df["n_empty_prompts"] / self.df["n_prompts"]
        self.df["miss_prompt_ratio"] = self.df["n_miss_prompts"] / self.df["n_prompts"]
        
        self.df["na_res_a_ratio"] = self.df["n_na_res_a"] / self.df["n_res_a"]
        self.df["empty_res_a_ratio"] = self.df["n_empty_res_a"] / self.df["n_res_a"]
        self.df["miss_res_a_ratio"] = self.df["n_miss_res_a"] / self.df["n_res_a"]
        self.df["na_res_b_ratio"] = self.df["n_na_res_b"] / self.df["n_res_b"]
        self.df["empty_res_b_ratio"] = self.df["n_empty_res_b"] / self.df["n_res_b"]
        self.df["miss_res_b_ratio"] = self.df["n_miss_res_b"] / self.df["n_res_b"]

        # 各種長さの統計を追加します
        for col, col_prefix in zip(["prompt", "response_a", "response_b"], ["p_len", "res_a_len", "res_b_len"]):
            self.df[f"{col_prefix}_med"] = self.df[col].apply(lambda x: np.median(list(self._len(s) for s in x)))  # 中央値を計算します
            self.df[f"{col_prefix}_std"] = self.df[col].apply(lambda x: np.std(list(self._len(s) for s in x)))  # 標準偏差を計算します

        # 有効な長さの平均を計算します
        self.df["p_len_eff_mean"] = self.df["p_len_sum"] / self.df["n_eff_prompts"]
        self.df["res_a_len_eff_mean"] = self.df["res_a_len_sum"] / self.df["n_eff_res_a"]
        self.df["res_b_len_eff_mean"] = self.df["res_b_len_sum"] / self.df["n_eff_res_b"]

        # 長さの差を計算します
        for stats in ["sum", "mean", "max", "med", "eff_mean"]:
            self.df[f"p_a_{stats}_diff"] = self.df[f"p_len_{stats}"] - self.df[f"res_a_len_{stats}"]
            self.df[f"p_b_{stats}_diff"] = self.df[f"p_len_{stats}"] - self.df[f"res_b_len_{stats}"]
            self.df[f"a_b_{stats}_diff"] = self.df[f"res_a_len_{stats}"] - self.df[f"res_b_len_{stats}"]
            
        # 特徴量列を定義します
        len_feature_a_col = ["res_a_len_sum","res_a_len_mean","res_a_len_max","res_a_len_sum_log","res_a_len_mean_log","res_a_len_max_log",
                     "res_a_len_med","res_a_len_std","res_a_len_eff_mean","p_a_sum_diff","p_a_mean_diff","p_a_max_diff","p_a_med_diff",
                     "p_a_eff_mean_diff"]
        
        len_feature_b_col = ["res_b_len_sum","res_b_len_mean","res_b_len_max","res_b_len_sum_log","res_b_len_mean_log","res_b_len_max_log",
                             "res_b_len_med","res_b_len_std","res_b_len_eff_mean","p_b_sum_diff","p_b_mean_diff","p_b_max_diff","p_b_med_diff",
                             "p_b_eff_mean_diff"]
        
        numerical_feature_columns = ["res_a_len_sum","res_a_len_mean","res_a_len_max","res_a_len_sum_log","res_a_len_mean_log","res_a_len_max_log",
                                     "res_a_len_med","res_a_len_std","res_a_len_eff_mean","p_a_sum_diff","p_a_mean_diff","p_a_max_diff","p_a_med_diff",
                                     "p_a_eff_mean_diff", "res_b_len_sum","res_b_len_mean","res_b_len_max","res_b_len_sum_log","res_b_len_mean_log","res_b_len_max_log",
                                     "res_b_len_med","res_b_len_std","res_b_len_eff_mean","p_b_sum_diff","p_b_mean_diff","p_b_max_diff","p_b_med_diff",
                                     "p_b_eff_mean_diff"]
        
        # ゼロで割らないように正規化します
        for col in numerical_feature_columns:
            if self.df[col].std() == 0:
                print(f"Warning: Standard deviation is zero for column {col}. Skipping normalization.")
            else:
                self.z_score_normalize([col])  # Zスコア正規化を適用します
                
        self.df = self.df.fillna(0)  # NaNを0で埋めます
        
        # 特徴量をリストに変換します
        len_features_a = self.df[len_feature_a_col].values.tolist()
        len_features_b = self.df[len_feature_b_col].values.tolist()

        return len_features_a, len_features_b  # 長さ特徴量を返します

# データ分割

以下のコードスニペットでは、class_label列の層化を使用して、既存のデータをトレーニングデータと検証データに分割します。

In [ ]:
from sklearn.model_selection import train_test_split  # パッケージをインポートします

# データフレームをトレーニングデータ（80%）と検証データ（20%）に分割します
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])  # class_label列をもとに層化分割を行います

# 前処理

In [ ]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset,  # 設定されたプリセットを使用して前処理器を作成します
    sequence_length=CFG.sequence_length,  # 設定されたシーケンス長を指定します
)

In [ ]:
def preprocess_fn(text, label=None, features_a=None, features_b=None):
    text = preprocessor(text)  # テキストを前処理します
    if features_a is not None:
        text['features_a'] = features_a  # features_aが指定されている場合、テキストに追加します
    if features_b is not None:
         text['features_b'] = features_b  # features_bが指定されている場合、テキストに追加します
    return (text, label) if label is not None else text  # ラベルがあれば、処理されたテキストとラベルを返します

# FGM（Fast Gradient Method）

In [ ]:
# FGMの摂動関数を追加します
def fgm_perturb(features, epsilon=1.0):
    # 摂動量を計算します。epsilonは摂動の割合です。
    perturbation = np.random.uniform(-1, 1, features.shape) * epsilon  # 特徴量の形状に基づいてランダムな摂動を生成します
    # 摂動を適用します
    return features + perturbation  # 特徴量に摂動を加えたものを返します

In [ ]:
# データ前処理関数を修正してFGM摂動を含めます
def preprocess_fn(text, label=None, features_a=None, features_b=None, is_fgm=False, epsilon=1.0):
    # テキストを前処理します
    text = preprocessor(text)
    if features_a is not None:
        if is_fgm:
            # FGMの場合、摺動を適用します
            features_a = fgm_perturb(features_a, epsilon)  # 特徴量AにFGM摂動を加えます
        text['features_a'] = features_a  # 特徴量Aをテキストに追加します
    if features_b is not None:
        if is_fgm:
            # FGMの場合、摺動を適用します
            features_b = fgm_perturb(features_b, epsilon)  # 特徴量BにFGM摂動を加えます
        text['features_b'] = features_b  # 特徴量Bをテキストに追加します
    return (text, label) if label is not None else text  # ラベルがあれば、処理されたテキストとラベルを返します

# AWP（Adversarial Weight Perturbation）

In [ ]:
# AWP摺動関数を定義します
def awp_perturb(model, epsilon=1e-4):
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            # 重みを取得します
            weights = layer.kernel
            # 摺動を計算します
            perturbation = tf.random.normal(weights.shape, stddev=epsilon)  # 標準偏差epsilonの正規分布から摺動を生成します
            # 摺動を適用します
            layer.kernel.assign_add(perturbation)  # 重みに摺動を加えます

# AWPコールバック関数を作成します
class AWPCallback(keras.callbacks.Callback):
    def __init__(self, epsilon):
        super(AWPCallback, self).__init__()  # 親クラスの初期化
        self.epsilon = epsilon  # 摺動の値を保存します

    def on_batch_begin(self, batch, logs=None):
        # 各バッチの開始時にAWP摺動を適用します
        awp_perturb(self.model, self.epsilon)  # モデルにAWP摺動を加えます

# DataLoader

以下のコードは、tf.data.Datasetを使用してデータ処理のための堅牢なデータフロー・パイプラインを設定します。

In [ ]:
def build_dataset_with_features(texts, labels=None, features_a=None, features_b=None, batch_size=32, is_fgm=False,  epsilon=1.0,
                                cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # 自動調整を有効にします
    if (features_a is not None) and (features_b is not None):
        # ラベルがない場合はテキストと特徴量のみ、ある場合はラベルも追加します
        slices = (texts, None, features_a, features_b) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3), features_a, features_b)  # スライスを作成します
    else:
        slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # スライスを作成します
    ds = tf.data.Dataset.from_tensor_slices(slices)  # テンソルのスライスからデータセットを生成します
    ds = ds.cache() if cache else ds  # キャッシュを使用するかどうかを設定します
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # 前処理関数を各要素に適用します
#     ds = ds.map(lambda x: preprocess_fn(x, features_a=features_a, features_b=features_b, is_fgm=is_fgm, epsilon=epsilon),
#                 num_parallel_calls=tf.data.AUTOTUNE)  # (コメントアウトされたコード)摺動を考慮した前処理
    opt = tf.data.Options()
    if shuffle:
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # シャッフルを有効にします
        opt.experimental_deterministic = False  # 結果の非決定性を許可します
    ds = ds.with_options(opt)  # オプションを適用します
    ds = ds.batch(batch_size, drop_remainder=False)  # バッチに分けます
    ds = ds.prefetch(AUTO)  # プリフェッチを使用してデータを事前に読み込む

    return ds  # データセットを返します

## トレーニング/検証データローダーの構築

In [ ]:
train_features_processor = DataFrameStatsProcessor(train_df.copy())  # トレーニングデータフレームのコピーを使用して特徴量処理器を作成します
train_features_a, train_features_b = train_features_processor.process_dataframe()  # トレーニングデータの特徴量を処理します
valid_features_processor = DataFrameStatsProcessor(valid_df.copy())  # 検証データフレームのコピーを使用して特徴量処理器を作成します
valid_features_a, valid_features_b = valid_features_processor.process_dataframe()  # 検証データの特徴量を処理します

In [ ]:
# トレーニングデータの準備
train_texts = train_df.options.tolist()  # トレーニングデータのテキストをリストに変換します
train_labels = train_df.class_label.tolist()  # トレーニングデータのラベルをリストに変換します
train_ds = build_dataset_with_features(train_texts, train_labels, train_features_a, train_features_b, 
                         batch_size=CFG.batch_size,  # 設定されたバッチサイズを使用します
                         shuffle=True)  # シャッフルを有効にします

# 検証データの準備
valid_texts = valid_df.options.tolist()  # 検証データのテキストをリストに変換します
valid_labels = valid_df.class_label.tolist()  # 検証データのラベルをリストに変換します
valid_ds = build_dataset_with_features(valid_texts, valid_labels, valid_features_a, valid_features_b, 
                         batch_size=CFG.batch_size,  # 設定されたバッチサイズを使用します
                         shuffle=False)  # シャッフルを無効にします
print(train_ds)  # トレーニングデータセットの情報を表示します

# 学習率スケジュール

学習率スケジューラの実装は、転移学習において重要です。

学習率は、lr_startから始まり、さまざまな手法を使用してlr_minまで徐々に減少します。これには以下が含まれます：

- step：階段状に学習率を段階的に減少させます。
- cos：コサイン曲線を利用して学習率を徐々に減少させます。
- exp：指数関数的に学習率を減少させます。

**重要性**：適切に構造化された学習率スケジュールは、モデルの効果的なトレーニングに不可欠であり、最適な収束を保証し、オーバーシュートや停滞などの問題を回避します。

In [ ]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6  # 学習率の開始値、最大値、最小値を設定します
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8  # 学習率のランプアップエポック、持続エポック、減衰率を設定します

    def lrfn(epoch):  # 学習率更新関数
        # エポックに応じて学習率を計算します
        if epoch < lr_ramp_ep: 
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start  # ランプアップ段階
        elif epoch < lr_ramp_ep + lr_sus_ep: 
            lr = lr_max  # 最大学習率を維持します
        elif mode == 'exp:': 
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min  # 指数的減衰
        elif mode == 'step': 
            lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)  # ステップ減衰
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs  # コサインの位相を計算します
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min  # コサイン曲線による減衰
        return lr  # 計算された学習率を返します
    
    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # 学習率コールバックを作成します

In [ ]:
lr_cb = get_lr_callback(CFG.batch_size, epochs=CFG.epochs)  # 設定されたバッチサイズとエポック数で学習率コールバックを取得します

# モデルチェックポイント

以下のコードは、トレーニング中にモデルの最良のチェックポイントを保存するコールバックを作成します。これは、提出時の推論に使用します。

In [ ]:
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',  # 検証ロスを監視します
                                          save_best_only=True,  # 最良のモデルのみを保存します
                                          save_weights_only=True,  # 重みのみを保存します
                                          mode='min')  # 最小化モードで監視します（ロスが最小のときに保存）

# メトリック

今回のコンペティションの評価指標は対数損失です。この尺度は数学的に次のように表されます：

$$
\text{Log Loss} = -\frac{1}{N} \sum_{i=1}^{N} \left( y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right)
$$

In [ ]:
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss", label_smoothing=0.1, from_logits=False)  # 対数損失を計算するためのメトリックを定義します。ラベルスムージングを適用します。

# モデリング

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

with strategy.scope():

    # すべての入力層を辞書にまとめます
    inputs = {
        "token_ids": keras.layers.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),  # トークンIDの入力
        "padding_mask": keras.layers.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),  # パディングマスクの入力
        "features_a": keras.layers.Input(shape=(14,), name="features_a", dtype=tf.float32),  # 特徴量Aの入力
        "features_b": keras.layers.Input(shape=(14,), name="features_b", dtype=tf.float32),  # 特徴量Bの入力
    }

    # DebertaV3Classifierバックボーンを作成します
    backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
        CFG.preset,
    )

    # response_aとresponse_bの作成方法を修正し、padding_maskを含めます
    response_a = {
        "token_ids": inputs["token_ids"][:, 0, :],  # 最初のトークンIDを取得
        "padding_mask": inputs["padding_mask"][:, 0, :]  # 最初のパディングマスクを取得
    }
    embed_a = backbone(response_a)  # バックボーンを通して埋め込みを取得します

    response_b = {
        "token_ids": inputs["token_ids"][:, 1, :],  # 2番目のトークンIDを取得
        "padding_mask": inputs["padding_mask"][:, 1, :]  # 2番目のパディングマスクを取得
    }
    embed_b = backbone(response_b)  # バックボーンを通して埋め込みを取得します
    
    # 数値特徴を埋め込みます
    len_features_a_embedding = keras.layers.Dense(512, activation='relu')(inputs["features_a"])  # 特徴量Aの数値埋め込み
    len_features_b_embedding = keras.layers.Dense(512, activation='relu')(inputs["features_b"])  # 特徴量Bの数値埋め込み
    
    # Flatten層を使用して数値特徴埋め込みを2次元テンソルに展開します
    flattened_len_features_a = keras.layers.Flatten()(len_features_a_embedding)  # Flattenによる変換
    flattened_len_features_b = keras.layers.Flatten()(len_features_b_embedding)  # Flattenによる変換
    
    embed_a = keras.layers.GlobalAveragePooling1D()(embed_a)  # グローバル平均プーリング
    embed_b = keras.layers.GlobalAveragePooling1D()(embed_b)  # グローバル平均プーリング
    embeds_text_features_a = keras.layers.Concatenate(axis=-1)([embed_a, flattened_len_features_a])  # テキスト埋め込みと数値特徴を結合
    embeds_text_features_b = keras.layers.Concatenate(axis=-1)([embed_b, flattened_len_features_b])  # テキスト埋め込みと数値特徴を結合
    
    # テキスト埋め込みと数値特徴埋め込みを結合します
    combined_embeds = keras.layers.Concatenate(axis=-1)([embeds_text_features_a, embeds_text_features_b])  # テキストと特徴を結合
    
    # モデルにL2正則化とDropoutを追加します
    combined_embeds = keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(combined_embeds)  # L2正則化
    combined_embeds = Dropout(0.05)(combined_embeds)  # Dropoutレイヤー、5%のニューロンをドロップ
    
    # 温度スケール関数を定義します
    def temperature_scale(logits, T=1.0):
        return logits / T  # ロジットを温度で割ります
    
    # 温度パラメータTを定義します
    T = 0.85
    # 温度スケーリングを適用します
    scaled_logits = temperature_scale(combined_embeds, T)  # 温度スケールを適用したロジット
    outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(scaled_logits)  # 出力層を定義します
    
    model = keras.Model(inputs, outputs)  # モデルを作成します
    
    # オプティマイザ、損失、メトリックでモデルをコンパイルします
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-6, clipnorm=1.0),  # Adamオプティマイザを使用
        loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1, from_logits=False),  # カテゴリカルクロスエントロピー損失
        metrics=[
            log_loss,  # 定義した対数損失
            keras.metrics.CategoricalAccuracy(name="accuracy"),  # 精度メトリック
        ],
    )
    
    # AWPコールバックをモデルのトレーニングに追加します
    awp_cb = AWPCallback(epsilon=1e-4)  # epsilon値は必要に応じて調整できます

### モデルの概要

In [ ]:
model.summary()  # モデルのサマリーを表示します。各層の出力形状やパラメータ数を確認できます。

# トレーニング

In [ ]:
# tryブロック内でエラー処理を実装します
try:
    history = model.fit(
        train_ds,  # トレーニングデータセット
        epochs=CFG.epochs,  # 設定されたエポック数で学習
        validation_data=valid_ds,  # 検証データセット
        callbacks=[lr_cb, ckpt_cb, awp_cb]  # 学習率コールバック、チェックポイントコールバック、AWPコールバックを追加
    )
except tf.errors.InvalidArgumentError as e:
    print(f"無効な引数エラーが発生しました：{e}")  # エラーが発生した場合、そのメッセージを表示します

## 最良モデルの読み込み

In [ ]:
model.load_weights('/kaggle/working/best_model.weights.h5')  # 最良モデルの重みを読み込みます

# 予測

In [ ]:
# FGM摺動を使用したデータセットでモデルを評価します
fgm_ds = build_dataset_with_features(train_texts, train_labels, train_features_a, train_features_b,
                                     is_fgm=True, epsilon=1.0)  # FGMを使用してデータセットを構築します
evaluation_results = model.evaluate(fgm_ds)  # 評価を実行します

print(f"FGM摺動データセットの評価結果: {evaluation_results}")  # FGM摺動データセットでの評価結果を表示します

In [ ]:
test_df_features_processor = DataFrameStatsProcessor(test_df)  # テストデータフレームの特徴量処理器を作成します
test_df_features_a, test_df_features_b = test_df_features_processor.process_dataframe()  # テストデータの特徴量を処理します

In [ ]:
test_texts = test_df.options.tolist()  # テストデータのテキストをリストに変換します
test_ds = build_dataset_with_features(test_texts, features_a=test_df_features_a, features_b=test_df_features_b,
                         batch_size=min(len(test_df), CFG.batch_size),  # テストデータの長さと設定されたバッチサイズの最小値を使用します
                         shuffle=False)  # シャッフルを無効にします
print(test_ds)  # テストデータセットの情報を表示します

In [ ]:
test_preds = model.predict(test_ds, verbose=1)  # テストデータセットに対してモデルの予測を実行します。進捗を表示します。

# 提出

In [ ]:
sub_df = test_df[["id"]].copy()  # テストデータから'id'列をコピーします
sub_df[CFG.class_names] = test_preds.tolist()  # 予測結果をコピーして新しいデータフレームに追加します
sub_df.to_csv("submission.csv", index=False)  # 提出用ファイルをCSV形式で保存します
sub_df.head()  # 提出データフレームの先頭を表示します